In [1]:
#Importing and loading the essential libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import rasterio
from rasterio.transform import rowcol
from scipy.stats import zscore
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

In [3]:
# Location where the tiff file that contains all climate varibles for each loaction is available 
tiff_path = "/Users/vinayakshanbhag/Downloads/TerraClimate_output.tiff"

# Function to load climate data of a dataset containing latitude and longitude 
# This is used to extract climate variables for both training set and validation set
def extract_climate(df, tiff_path):
    with rasterio.open(tiff_path) as src:
        bands = [src.read(i) for i in range(1, src.count + 1)]
        extracted = []
        for _, row in df.iterrows():
            lon, lat = row["Longitude"], row["Latitude"]
            try:
                r, c = rowcol(src.transform, lon, lat)
                pixel_vals = [bands[b][r, c] for b in range(len(bands))]
            except:
                pixel_vals = [np.nan] * len(bands)
            extracted.append(pixel_vals)

    climate_vars = ['tmin', 'tmax', 'vap', 'ppt', 'srad', 'ws',
                    'aet', 'pet', 'q', 'def', 'soil', 'swe', 'pdsi', 'vpd']
    climate_df = pd.DataFrame(extracted, columns=climate_vars)
    climate_df["Latitude"] = df["Latitude"].values
    climate_df["Longitude"] = df["Longitude"].values
    return climate_df

In [5]:
# Load Datasets
train_df = pd.read_csv("/Users/vinayakshanbhag/Downloads/Training_Data.csv")
val_df = pd.read_csv("/Users/vinayakshanbhag/Downloads/Validation_Template.csv")

train_climate = extract_climate(train_df, tiff_path)
val_climate = extract_climate(val_df, tiff_path)

# Ensure df_climate is correctly processed from the TIFF file
print("Climate (Training) Data Shape:", train_climate.shape)
print(train_climate.head())

Climate (Training) Data Shape: (6312, 16)
        tmin        tmax  vap         ppt  srad   ws        aet         pet  \
0  53.500000   65.300003 -4.5  115.500000  49.7  2.5  16.700001  200.799149   
1  24.800001  110.800003 -3.9  143.100006  26.0  1.3   2.500000  217.699799   
2  51.299999   28.200001 -3.8  115.099998  69.9  3.5  68.800003  204.000031   
3  41.000000   67.300003 -4.7  120.700005  45.0  2.3  11.300000  204.400146   
4  58.900002   29.500000 -4.8  109.500000  71.1  3.6  43.000000  189.203964   

     q        def       soil    swe  pdsi  vpd   Latitude   Longitude  
0  0.0  23.900000  12.599999  1.233  0.81  3.6 -34.027900  150.771000  
1  0.0  24.400000  10.700000  0.938  1.28  3.1 -34.821595  147.193697  
2  0.0  21.400000   8.099999  0.942  0.78  3.2 -36.617759  146.882941  
3  0.0  20.199999   8.000000  0.951  0.70  4.4 -37.470900  144.744000  
4  0.0  18.900000   9.900000  1.096  0.50  5.6 -38.400153  145.018560  


In [7]:
# Merge datasets to train the model on this datset that contains climate variables and occurences for each latitude and longitude values
# Rounding to 4 decimal places so that its easier
train_df["Latitude"] = train_df["Latitude"].round(4)
train_df["Longitude"] = train_df["Longitude"].round(4)
train_climate["Latitude"] = train_climate["Latitude"].round(4)
train_climate["Longitude"] = train_climate["Longitude"].round(4)

val_df["Latitude"] = val_df["Latitude"].round(4)
val_df["Longitude"] = val_df["Longitude"].round(4)
val_climate["Latitude"] = val_climate["Latitude"].round(4)
val_climate["Longitude"] = val_climate["Longitude"].round(4)

# Merge and drop values with null values
train_merged = pd.merge(train_df, train_climate, on=["Latitude", "Longitude"]).dropna()
val_merged = pd.merge(val_df, val_climate, on=["Latitude", "Longitude"]).dropna()

# Ensure train_merged is correctly processed from the TIFF file
print("Climate Data Shape:", train_merged.shape)
print(train_merged.head())

Climate Data Shape: (6419, 17)
   Latitude  Longitude  Occurrence Status       tmin        tmax  vap  \
0  -34.0279   150.7710                  1  53.500000   65.300003 -4.5   
1  -34.8216   147.1937                  1  24.800001  110.800003 -3.9   
2  -36.6178   146.8829                  0  51.299999   28.200001 -3.8   
3  -37.4709   144.7440                  1  41.000000   67.300003 -4.7   
4  -38.4002   145.0186                  1  58.900002   29.500000 -4.8   

          ppt  srad   ws        aet         pet    q        def       soil  \
0  115.500000  49.7  2.5  16.700001  200.799149  0.0  23.900000  12.599999   
1  143.100006  26.0  1.3   2.500000  217.699799  0.0  24.400000  10.700000   
2  115.099998  69.9  3.5  68.800003  204.000031  0.0  21.400000   8.099999   
3  120.700005  45.0  2.3  11.300000  204.400146  0.0  20.199999   8.000000   
4  109.500000  71.1  3.6  43.000000  189.203964  0.0  18.900000   9.900000   

     swe  pdsi  vpd  
0  1.233  0.81  3.6  
1  0.938  1.28  3

In [9]:
# Outlier & Missing Handling
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]

def remove_outliers_z(data, column, threshold=3.5):
    z_scores = np.abs(zscore(data[column]))
    return data[z_scores < threshold]

# Apply on selected variables
for col in ["aet", "soil"]:
    train_merged = remove_outliers_iqr(train_merged, col)
for col in ["pdsi", "vap", "ws"]:
    train_merged = remove_outliers_z(train_merged, col)

In [11]:
# Drop highly correlated variables threshold > 0.9
drop_vars = ['pet', 'ppt', 'q', 'vpd']
X = train_merged.drop(columns=["Occurrence Status", "Latitude", "Longitude"] + drop_vars)
y = train_merged["Occurrence Status"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_val = val_merged.drop(columns=["Latitude", "Longitude"] + drop_vars)
X_val_scaled = scaler.transform(X_val)

In [13]:
# Split for local evaluation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

In [8]:
# Model 1: Default RF

# Train model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f" Random Forest Accuracy: {accuracy:.4f}")
print("\n Confusion Matrix:\n", conf_matrix)
print("\n Classification Report:\n", class_report)

 Random Forest Accuracy: 0.7804

 Confusion Matrix:
 [[320 151]
 [109 604]]

 Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.68      0.71       471
           1       0.80      0.85      0.82       713

    accuracy                           0.78      1184
   macro avg       0.77      0.76      0.77      1184
weighted avg       0.78      0.78      0.78      1184



In [9]:
# Predict on validation set
val_rfpreds = rf_model.predict(X_val_scaled)
val_merged["Occurrence Status"] = val_rfpreds
val_merged[["Latitude", "Longitude", "Occurrence Status"]].to_csv("Predicted_RF.csv", index=False)
print(" Saved as: Predicted_RF.csv")

 Saved as: Predicted_RF.csv


In [10]:
# Model 2: Tuned XGBoost

xgb_model = XGBClassifier(
    subsample=1.0,
    reg_lambda=5,
    reg_alpha=1,
    n_estimators=100,
    max_depth=8,
    learning_rate=0.1,
    gamma=0,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_preds_test = xgb_model.predict(X_test)

# Evaluate
print(" Tuned XGBoost Performance:")
print(" Accuracy:", accuracy_score(y_test, xgb_preds_test))
print(" Classification Report:", classification_report(y_test, xgb_preds_test))

 Tuned XGBoost Performance:
 Accuracy: 0.768581081081081
 Classification Report:               precision    recall  f1-score   support

           0       0.74      0.65      0.69       471
           1       0.78      0.85      0.82       713

    accuracy                           0.77      1184
   macro avg       0.76      0.75      0.75      1184
weighted avg       0.77      0.77      0.77      1184



/Users/vinayakshanbhag/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:07:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [11]:
# Predict on validation set
val_preds_xgb = xgb_model.predict(X_val_scaled)
val_merged["Occurrence Status"] = val_preds_xgb
val_merged[["Latitude", "Longitude", "Occurrence Status"]].to_csv("Predicted_XGB_Final.csv", index=False)
print(" Saved as: Predicted_XGB_Final.csv")

 Saved as: Predicted_XGB_Final.csv


In [12]:
# Model 3: Tuned RF

# Train model
rftuned1_model = RandomForestClassifier(
    n_estimators=290,
    max_depth=10,
    max_features='log2',
    min_samples_leaf=2,
    min_samples_split=2,
    bootstrap=False,
    random_state=42,
    n_jobs=-1
)
rftuned1_model.fit(X_train, y_train)

# Predict
rftuned1_preds_test = rftuned1_model.predict(X_test)

# Evaluate
print(" Tuned 1 Random Forest Performance:")
print(" Accuracy:", accuracy_score(y_test, rftuned1_preds_test))
print(" Classification Report:", classification_report(y_test, rftuned1_preds_test))

 Tuned 1 Random Forest Performance:
 Accuracy: 0.777027027027027
 Classification Report:               precision    recall  f1-score   support

           0       0.80      0.59      0.68       471
           1       0.77      0.90      0.83       713

    accuracy                           0.78      1184
   macro avg       0.78      0.74      0.75      1184
weighted avg       0.78      0.78      0.77      1184



In [13]:
# Predict on validation set
val_preds_rf = rftuned1_model.predict(X_val_scaled)
val_merged["Occurrence Status"] = val_preds_rf
val_merged[["Latitude", "Longitude", "Occurrence Status"]].to_csv("Predicted_RF_Final.csv", index=False)

In [14]:
# Model 4: Tuned RF with RandomizedSearchCV

# Define parameter grid for tuning
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Initialize the base model
rf = RandomForestClassifier(random_state=42)

# Perform randomized search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='f1',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit on your cleaned training data (X_train_scaled and y_train)
random_search.fit(X_train, y_train)

# Extract the best model
best_rf = random_search.best_estimator_

# Predict on holdout test set
rf_test_preds = best_rf.predict(X_test)

# Evaluate
print(" Tuned Random Forest (via RandomizedSearchCV)")
print(" Best Hyperparameters:", random_search.best_params_)
accuracy = accuracy_score(y_test, rf_test_preds)
print(f"\n Accuracy: {accuracy:.4f}")
print("\n Classification Report:")
print(classification_report(y_test, rf_test_preds))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
 Tuned Random Forest (via RandomizedSearchCV)
 Best Hyperparameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 174}

 Accuracy: 0.7838

 Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.68      0.71       471
           1       0.80      0.86      0.83       713

    accuracy                           0.78      1184
   macro avg       0.78      0.77      0.77      1184
weighted avg       0.78      0.78      0.78      1184



In [15]:
# Predict on validation set
val_rf_preds = best_rf.predict(X_val_scaled)
val_merged["Occurrence Status"] = val_rf_preds
val_merged[["Latitude", "Longitude", "Occurrence Status"]].to_csv("Predicted_Data_RF_Tuned_CV_.csv", index=False)

In [15]:
# Model 5: RF Advancedtuning + Expanded Hyperparameter Grid + StratifiedKFold + Class Weights (Best Model)

# Define enhanced parameter grid
param_dist = {
    'n_estimators': randint(150, 500),
    'max_depth': [None, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 'auto'],
    'bootstrap': [True, False]
}

# Use Stratified K-Fold for CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize RF with class weights
rf = RandomForestClassifier(class_weight="balanced", random_state=42)

# Randomized Search CV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='f1',
    cv=cv,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit
random_search.fit(X_train, y_train)
best_rf_advanced = random_search.best_estimator_

# Evaluate
test_preds = best_rf_advanced.predict(X_test)
print(" Best Parameters:", random_search.best_params_)
print(" Accuracy:", accuracy_score(y_test, test_preds))
print(" Classification Report:", classification_report(y_test, test_preds))

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Users/vinayakshanbhag/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
70 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
38 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/vinayakshanbhag/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/vinayakshanbhag/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/vinayakshanbhag/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 637, in _validate_params
    valida

 Best Parameters: {'bootstrap': True, 'max_depth': 25, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 480}
 Accuracy: 0.7829391891891891
 Classification Report:               precision    recall  f1-score   support

           0       0.73      0.71      0.72       471
           1       0.81      0.83      0.82       713

    accuracy                           0.78      1184
   macro avg       0.77      0.77      0.77      1184
weighted avg       0.78      0.78      0.78      1184



In [17]:
# Predict on validation
val_preds = best_rf_advanced.predict(X_val_scaled)
val_merged["Occurrence Status"] = val_preds
val_merged[["Latitude", "Longitude", "Occurrence Status"]].to_csv("Predicted_RF_Tuned_Advanced.csv", index=False)
print(" Saved as: Predicted_RF_Tuned_Advanced.csv")

 Saved as: Predicted_RF_Tuned_Advanced.csv


In [18]:
# Define enhanced parameter grid
param_dist = {
    'n_estimators': randint(150, 500),
    'max_depth': [None, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Use Stratified K-Fold for CV
cvs = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize RF with class weights
rf1 = RandomForestClassifier(class_weight="balanced", random_state=42)

# Randomized Search CV
random_search1 = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='f1',
    cv=cv,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit
random_search1.fit(X_train, y_train)
best_rf_advanced1 = random_search1.best_estimator_

# Evaluate
test_preds1 = best_rf_advanced1.predict(X_test)
print(" Best Parameters:", random_search1.best_params_)
print(" Accuracy:", accuracy_score(y_test, test_preds1))
print(" Classification Report:", classification_report(y_test, test_preds1))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
 Best Parameters: {'bootstrap': True, 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 316}
 Accuracy: 0.7837837837837838
 Classification Report:               precision    recall  f1-score   support

           0       0.75      0.68      0.72       471
           1       0.80      0.85      0.83       713

    accuracy                           0.78      1184
   macro avg       0.78      0.77      0.77      1184
weighted avg       0.78      0.78      0.78      1184



In [19]:
# Predict on validation
val_preds1 = best_rf_advanced1.predict(X_val_scaled)
val_merged["Occurrence Status"] = val_preds1
val_merged[["Latitude", "Longitude", "Occurrence Status"]].to_csv("Predicted_RF_Tuned_xyz.csv", index=False)
print(" Saved as: Predicted_RF_Tuned_xyz.csv")

 Saved as: Predicted_RF_Tuned_xyz.csv


In [20]:
train_merged.to_csv("Training_Data_Climate_Merged.csv", index=False)
